## Singular Value Decomposition (SVD)

SVD is the core algorithm behind LSA.
<br>
As an instance, start with a corpus containing 11 documents and a vocabulary of 6 words:

In [1]:
from nlpia.book.examples.ch04_catdog_lsa_sorted import lsa_models, prettify_tdm

100%|██████████| 263/263 [00:00<00:00, 199367.78it/s]


In [2]:
bow_svd, tfidf_svd = lsa_models()
prettify_tdm(**bow_svd)

100%|██████████| 263/263 [00:00<00:00, 169213.37it/s]


,cat,dog,apple,lion,nyc,love,text
0,,,1,,1,,NYC is the Big Apple.
1,,,1,,1,,NYC is known as the Big Apple.
2,,,,,1,1,I love NYC!
3,,,1,,1,,I wore a hat to the Big Apple party in NYC.
4,,,1,,1,,Come to NYC. See the Big Apple!
5,,,1,,,,Manhattan is called the Big Apple.
6,1,,,,,,New York is a big city for a small cat.
7,1,,,1,,,"The lion, a big cat, is the king of the jungle."
8,1,,,,,1,I love my pet cat.
9,,,,,1,1,I love New York City (NYC).


The above matrix shown is a document-term matrix (dtm) where each row is a vector of the BOW for a document.
* Interpretation - The sorting algorithm and the limited vocabulary created several identical BOW vectors (NYC, apple)
* SVD operations - SVD should be able to notice this and allocate a topic to such pair of words


Furthermore, we can use SVD on the term-document matrix (tdm) - the transposition of a dtm - where it can work on TF-IDF matrices or any other vector space model:

In [3]:
tdm = bow_svd['tdm']
tdm

,0,1,2,3,4,5,6,7,8,9,10
cat,0,0,0,0,0,0,1,1,1,0,1
dog,0,0,0,0,0,0,0,0,0,0,1
apple,1,1,0,1,1,1,0,0,0,0,0
lion,0,0,0,0,0,0,0,1,0,0,0
nyc,1,1,1,1,1,0,0,0,0,1,0
love,0,0,1,0,0,0,0,0,1,1,0


SVD is an algorithm for decomposing any matrix into three factors - three matrices that can be multiplied together to recreate the original matrix.
* Purpose - The three matrix factors computed with SVD contain some convenient mathematical properties we can exploit for dimension reduction and LSA. LSA will be used to figure out topics (group of related words) need to be

Whether we run SVD on a word vector represention (BOW or TFIDF term-document matrices), SVD will find combinations of words that belong together 
* Process - SVD finds those co-occurring words by calculating the correlation between the columns (terms) of our term-document matrix
* Computation - SVD simultaneously finds the correlation of term use between documents and the correlation of documents with each other, additionally computing the linear combinations of terms that have the greatest variation across the corpus
* Filtering and dimensions reduction - We'll only keep those topics that retain the most information i.e. the most variance in our corpus
* Transformation - SVD gives us the linear transformation (rotation) of our term-document vectors to convert those vectors into shorter topic vectors for each document

SVD will group together terms that have high correlation with each other (given they appear in the same documents together frequently) and also vary together a lot over the set of documents.
<br>
These linear combinations of words are seen as 'topics'.
<br>
These topics that turn our BOW/TF-IDF vectors into topic vectors that tell us the topics a document is about.
<br>
A topic vector provides a summarization or generalization of what the document is about.

In mathematical terms, SVD appears like this:
<br>
<br>
$W_{mxn}$ &#8594; $U_{mxp}$$S_{pxp}$$V_{pxn}$$^{T}$

* $m$ - The number of terms in one's vocabulary 
* $n$ - The number of documents in a corpus 
* $p$ - The number of topics in a corpus i.e. the number of words 

We want to eventually end up with fewer topics than words, so we can use those topic vectors (rows of topic-document matrix) as a reduced-dimension representation of the original TF-IDF vectors. 
<br>
But currently in this example as a first stage, we retain all the dimensions in our matrices.
<br>
It's key to uncover what these three matrices (U, S and V) look like.

### $U$ - left singular values 

The $U$ matrix contains the term-topic matrix that tells us about 'the company a word keeps' - the core matrix for semantic analysis in NLP.
* **Left singular matrix** - The $U$ matrix is also known to be the 'left singular matrix' as it contains row vectors that should be multiplied by a matrix of column vectors from the left 
* Representation - $U$ is the cross-correlation between words and topics based on word co-occurrence in the same document

$U$ is a square matrix until we start truncating it (deleting columns). 
* Dimensions - $U$ contains the same number of rows and columns as we have words in our vocabulary ($m$): 6. Given we haven't truncated this matrix yet, the number of topics ($p$) remains 6

In [4]:
import numpy as np 
import pandas as pd 
U, s, Vt = np.linalg.svd(tdm)
pd.DataFrame(U, index=tdm.index).round(2)

,0,1,2,3,4,5
cat,-0.04,0.83,-0.38,0.00,0.11,-0.38
dog,-0.00,0.21,-0.18,-0.71,-0.39,0.52
apple,-0.62,-0.21,-0.51,0.00,0.49,0.27
lion,-0.00,0.21,-0.18,0.71,-0.39,0.52
nyc,-0.75,-0.00,0.24,-0.00,-0.52,-0.32
love,-0.22,0.42,0.69,0.00,0.41,0.37


The $U$ matrix contains all the topic vectors for each word in our corpus as columns.
* Conversion - Such matrix can be used as a transformation to convert a word-document vector (TF-IDF/BOW vector) into a topic-document vector
* Weight computation - We multiply our topic vector $U$ matrix by any word-document column vector to get a new topic-document vector, given that the weights/scores in each cell of the $U$ matrix represent how important each word is to each topic 

In essence, the $U$ matrix in terms of SVD provides us with everything needed to map word frequencies to topics

### $S$ - singular values

The Sigma or $S$ matrix contains the topic 'singular values' in a square diagonal matrix 
* Singular values - such values tell us how much information is captured by each dimension in our new semantic (topic) vector space
* Diagonal matrix - contains only nonzero values along the diagonal from the upper left to the right. Everywhere else will contain zeros 

In [5]:
s.round(1)

array([3.1, 2.2, 1.8, 1. , 0.8, 0.5])

In [6]:
S = np.zeros((len(U), len(Vt)))
pd.np.fill_diagonal(S, s)

In [7]:
pd.DataFrame(S).round(1)

,0,1,2,3,4,5,6,7,8,9,10
0,3.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,2.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0


As with the $U$ matrix, our $S$ matrix for the 6 word/topic corpus contains 6 rows ($p$), but it contains many more columns ($n$) filled with zeros.
* Process - It needs a column for every document so we can multiply it by $V^T$ (document-document matrix). As we haven't yet reduced the dimensionality via truncating this diagonal matrix, we have as many topics ($p$) as we have terms in our vocabulary ($m$), which is six in this case
* Dimension construction - the dimensions (topics) are constructed such that the first dimension contains the most information ('explained variance') about our corpus
* Truncating - When it comes to truncating the topic model, we can begin zeroing out the dimensions at the lower right and work our way up to the left. We can stop zeroing out these sigular values when the error in our topic model starts to contribute significantly to the overall NLP pipeline error


### $V^T$ - right singular vectors 


The $V^T$ matrix contains the 'right singular vectors' as the columns of the document-document matrix.
* Purpose - The $V^T$ matrix gives us the shared meaning between documents, since it measures how often documents use the same topics in our new semantic model of documents 
* Representation - It has the same number of rows ($p$) and columns as we have documents in our small corpus, which in this case is 11

In [8]:
pd.DataFrame(Vt).round(2)

,0,1,2,3,4,5,6,7,8,9,10
0,-0.44,-0.44,-0.31,-0.44,-0.44,-0.20,-0.01,-0.01,-0.08,-0.31,-0.01
1,-0.09,-0.09,0.19,-0.09,-0.09,-0.09,0.37,0.47,0.56,0.19,0.47
2,-0.16,-0.16,0.52,-0.16,-0.16,-0.29,-0.22,-0.32,0.17,0.52,-0.32
3,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.71,0.00,-0.00,-0.71
4,-0.04,-0.04,-0.14,-0.04,-0.04,0.58,0.13,-0.33,0.62,-0.14,-0.33
5,-0.09,-0.09,0.10,-0.09,-0.09,0.51,-0.73,0.27,-0.01,0.10,0.27
6,-0.57,0.21,0.11,0.33,-0.31,0.34,0.34,-0.00,-0.34,0.23,0.00
7,-0.32,0.47,0.25,-0.63,0.41,0.07,0.07,0.00,-0.07,-0.18,0.00
8,-0.50,0.29,-0.20,0.41,0.16,-0.37,-0.37,-0.00,0.37,-0.17,0.00
9,-0.15,-0.15,-0.59,-0.15,0.42,0.04,0.04,0.00,-0.04,0.63,-0.00


As with the $S$ matrix, we can ignore the $V^T$ matrix whenever we're transforming new word-document vectors into our topic vector space.
<br>
The $V^T$ matrix may only be used once more to check the accuracy of topic vector for recreating the original word-document vectors that we used to 'train' it.

### SVD orientation

When we perform the SVD linear algebra directly, the matrix needs to be in the format (transposed if needed) consisting of a term-document matrix.
<br>
ML packages such as `sklearn` require a document-term matrix in terms of a training set instead, which in this case would mean:
* Rows - Each row in the sample-feature matrix for a ML sample is a document (textual data like SMS messages, reviews, emails etc.)
* Columns - Each column represents a word or feature (tokens) of those documents 

### Truncating topics 

Given the above notes, we have a way to constrauct a topic model - a way to transform word frequency vectors into topic weight vectors.
<br>
Although, given we have just as many topics as words, our vector space model has just as many dimensions as the original BOW vectors.
<br>
Carrying out this process means constructing some new words and calling them 'topics' as they each combine words together in various ratios.
<br>
We maintain the $U$ matrix given that it's already arranged so that the most important topics (with the largest singular values) are on the left - hence we can ignore the $S$ matrix.

To begin this truncation process and in effect reducing the number of dimensions, we can loop off columns on the right-hand side of the $U$ matrix.
* Accuracy/optimisation - To decide how many topics will be enough to capture the essence of a document, we need to measure the accuracy of LSA, which can be performed by indentifying how accurately we can recreate a term-document matrix from a topic-document  matrix 

In [9]:
err = [] 
for numdim in range(len(s), 0, -1):
    S[numdim -1, numdim - 1] = 0 
    reconstructed_tdm = U.dot(S).dot(Vt)
    err.append(np.sqrt(((reconstructed_tdm - tdm).values.flatten() ** 2).sum() / np.product(tdm.shape)))


In [10]:
np.array(err).round(2)

array([0.06, 0.12, 0.17, 0.28, 0.39, 0.55])

As seen among the elements in the array, when reconstructing a term-document matrix for our 11 documents using the singular vectors, the more we truncate, the more the error grows.

<img src="img-vects/lsa-tdm-accuracy-dimension-reduction.png" alt="LSA model accuracy shown to be decreasing given such tdm reconstruction as the number of dimensions are ignored/removed" width="400" height='250'/>

Figure 1 NLPIA Lane, Howard and Hapke (2019) chp 4.3.5 pp. 330 Apple iBooks.

Figure 1 illustrates such phenomena where accuracy drops as one excludes more and more dimensions in a given topic model.
* Vector representation - The accuracy drop is quite similar between either BOW/TF-IDF vectors used for the model, although TF-IDF vectors will perform slightly better if we plan to retain only a few topics in such a model

The SVD algorithm behind LSA 'notices' if words are always used together and puts them together in a topic - exploiting a gain of few dimensions at no extra cost. 
* Other considerations - Even if a topic model isn't going to be used in a pipeline, LSA (SVD) can be a great way to compress one's word-document matrices and identify potential compound words or n-grams for a given pipeline